In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, SpatialDropout1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from glob import glob
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import CountVectorizer



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#files = glob('reviews/*.csv')
#data=pd.concat((pd.read_csv(file) for file in files))

data = pd.read_csv('/content/drive/My Drive/full_data.csv')

In [4]:
data_mod = data.drop(["movie_id","date","helpful","helpful_out_of"],axis=1)
final_data = data_mod.dropna()

In [5]:
final_data["rating"] = np.where(final_data['rating'] > 6.0, 1 , 0)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
reviews = final_data['review']
labels = final_data['rating']

In [7]:
# tokenize the dataset corpus, delete uncommon words such as names, etc.
tokenizer = Tokenizer(num_words=10000, oov_token=None)
tokenizer.fit_on_texts(reviews)
X = tokenizer.texts_to_sequences(reviews)
X, y = np.array(X), np.array(labels)
    # pad sequences with 0's
X = pad_sequences(X, maxlen=100)
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size=0.25, random_state=1)

In [8]:
lrr= ReduceLROnPlateau(monitor='val_accuracy', factor=.01,  patience=3, min_lr=1e-5)

In [9]:
# vector space embeddings
n_dims = 64
n_unique_words =10000
# RNN layer architecture
n_rnn = 128


In [10]:
model = Sequential()
model.add(Embedding(n_unique_words,n_dims,input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(n_rnn,dropout=0.2))
model.add(Dense(1,activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 64)           640000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 100, 64)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 738,945
Trainable params: 738,945
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, Y_train, batch_size=32, epochs=15, verbose=1, validation_data=(X_val, Y_val), callbacks=[lrr])

Epoch 1/15
39452/39452 [==============================] - 588s 15ms/step - loss: 0.3250 - accuracy: 0.8614 - val_loss: 0.2926 - val_accuracy: 0.8766
Epoch 2/15
39452/39452 [==============================] - 596s 15ms/step - loss: 0.2850 - accuracy: 0.8808 - val_loss: 0.2797 - val_accuracy: 0.8837
Epoch 3/15
39452/39452 [==============================] - 586s 15ms/step - loss: 0.2722 - accuracy: 0.8871 - val_loss: 0.2776 - val_accuracy: 0.8856
Epoch 4/15
39452/39452 [==============================] - 579s 15ms/step - loss: 0.2643 - accuracy: 0.8912 - val_loss: 0.2709 - val_accuracy: 0.8879
Epoch 5/15
39452/39452 [==============================] - 571s 14ms/step - loss: 0.2595 - accuracy: 0.8936 - val_loss: 0.2700 - val_accuracy: 0.8889
Epoch 6/15
39452/39452 [==============================] - 563s 14ms/step - loss: 0.2558 - accuracy: 0.8958 - val_loss: 0.2693 - val_accuracy: 0.8891
Epoch 7/15
39452/39452 [==============================] - 562s 14ms/step - loss: 0.2524 - accuracy: 0.8972